In [2]:
import pandas as pd

In [7]:
df = pd.read_json('cleaned_comments.json')

In [18]:
df

,rating,text,s
0,5,Онуку дуже сподобалось. Харчування для онука ц...,Positive
1,5,Клвсна. Малюку сподобалось. Замовляємо постійн...,Positive
2,5,"Добре підходить для густих каш і сумішей! + М,...",Positive
3,5,Чудова соска. Якість як завжди у Авент на найв...,Positive
4,3,з серії бебі преміум ця напевне зайшла найменш...,Negative
...,...,...,...
1507,5,Ніблер став для мене дуже приємним відкриттям....,Positive
1508,5,"Дуже смачне пюре, дитині сподобалось, натураль...",Positive
1509,5,"Серед виробників, які ми перепробували, а їх н...",Positive
1510,3,"нормальна каша, мала їсть",Negative


При выборе, какие данные парсить решил попробовать простую идею. В отделе "Для найменших" сидят молодые мамочки у которых есть свободное время (они домохозяйки) плюс повышенная мотвация оставлять коментарии т.к. качество продукта касается их лично. По-итогу только эту идею я и успел проверить т.к. парсинг Розетки оказался более времязатратным, чем я ожидал. Идея не сработала, оказалось, что коментариев они оставляют существенно меньше, чем люди в отделе, например, с персональной электроникой.
Парсил скриптом `script.py` 

В итоге вышло 7500 коментариев всего и только 1500 из них были на украинском.
Это количество абсолютно недостаточное для построения какой-либо вменяемой системы классификации.

In [11]:
df['s'] = df['rating'].apply(lambda x: 'Positive' if x > 4 else 'Negative')
df.groupby(['s']).count()

,rating,text
s,,
Negative,379,379
Positive,1133,1133


In [14]:
test_df = df.sample(frac=0.3)
train_df = df.drop(test_df.index)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression, Perceptron
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import pymorphy2
import time

In [25]:
clf = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB(fit_prior=False))])
clf = clf.fit(train_df.text, train_df.s)
test_df['Predicted'] = clf.predict(test_df.text)

print(classification_report(test_df.s, test_df.Predicted))

              precision    recall  f1-score   support

    Negative       0.63      0.38      0.47       110
    Positive       0.82      0.93      0.87       344

    accuracy                           0.80       454
   macro avg       0.73      0.65      0.67       454
weighted avg       0.78      0.80      0.78       454



In [21]:
class BigramCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(BigramCountVectorizer, self).build_analyzer()
        def bigram_analyzer(doc):
            tokens = analyzer(doc)
            return [' '.join(tokens[i:i+2]) for i in range(len(tokens)-1)]
        return bigram_analyzer

In [26]:
clf2 = Pipeline([('vect', BigramCountVectorizer()), ('clf', MultinomialNB(fit_prior=False))])
clf2 = clf2.fit(train_df.text, train_df.s)
test_df['Predicted'] = clf2.predict(test_df.text)

print(classification_report(test_df.s, test_df.Predicted))

              precision    recall  f1-score   support

    Negative       0.52      0.61      0.56       110
    Positive       0.87      0.82      0.84       344

    accuracy                           0.77       454
   macro avg       0.69      0.71      0.70       454
weighted avg       0.78      0.77      0.77       454



Я сделал только одну итерацию над обычным BoW, добавил биграмы, никакого сущестенного прироста в качестве не 
обнаружил. 
Но могу предсказать какие выводы можно было бы сделать по итогам этих экспериментов:
1. Линейные класификаторы будут показывать похожие результаты.
2. Нужно стараться собирать бОльшие датасет с как можно более разнообразными примерами.
3. Можно искуственно множить количество сэмплов для редких классов. Это должно улучшать f1 score и recall
4. Существенно помогла бы отчистка коментариев от кейсов где человек просто задаёт вопрос. 